## Linjär och Polynomisk Regression för Pumpapris - Lektion 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografik av Dasani Madipalli</figcaption>


<!--![Infografik av Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->

#### Introduktion

Hittills har du utforskat vad regression är med hjälp av exempeldata från pumpapris-datasetet som vi kommer att använda genom hela denna lektion. Du har också visualiserat det med hjälp av `ggplot2`.💪

Nu är du redo att fördjupa dig i regression för maskininlärning. I denna lektion kommer du att lära dig mer om två typer av regression: *grundläggande linjär regression* och *polynomisk regression*, tillsammans med lite av matematiken bakom dessa tekniker.

> Genom hela denna kurs antar vi minimal kunskap om matematik och strävar efter att göra det tillgängligt för studenter från andra områden. Håll utkik efter anteckningar, 🧮 matematiska inslag, diagram och andra lärverktyg som hjälper till att förstå.

#### Förberedelse

Som en påminnelse, du laddar denna data för att kunna ställa frågor om den.

-   När är den bästa tiden att köpa pumpor?

-   Vilket pris kan jag förvänta mig för en låda med miniatyrpumpor?

-   Bör jag köpa dem i halv-bushelkorgar eller i 1 1/9 bushel-lådor? Låt oss gräva djupare i denna data.

I föregående lektion skapade du en `tibble` (en modern omarbetning av data frame) och fyllde den med en del av den ursprungliga datasetet, där du standardiserade prissättningen per bushel. Genom att göra det kunde du dock bara samla in cirka 400 datapunkter och endast för höstmånaderna. Kanske kan vi få lite mer detaljer om datans natur genom att städa upp den mer? Vi får se... 🕵️‍♀️

För denna uppgift behöver vi följande paket:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) är en [samling av R-paket](https://www.tidyverse.org/packages) designade för att göra dataanalys snabbare, enklare och roligare!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) är ett [ramverk av paket](https://www.tidymodels.org/packages/) för modellering och maskininlärning.

-   `janitor`: [janitor-paketet](https://github.com/sfirke/janitor) erbjuder enkla verktyg för att undersöka och städa smutsig data.

-   `corrplot`: [corrplot-paketet](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) erbjuder ett visuellt utforskande verktyg för korrelationsmatriser som stödjer automatisk omordning av variabler för att hjälpa till att upptäcka dolda mönster bland variabler.

Du kan installera dem med:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Skriptet nedan kontrollerar om du har de paket som krävs för att slutföra denna modul och installerar dem åt dig om de saknas.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Vi kommer senare att ladda dessa fantastiska paket och göra dem tillgängliga i vår nuvarande R-session. (Detta är bara för illustration, `pacman::p_load()` har redan gjort det åt dig)

## 1. En linjär regressionslinje

Som du lärde dig i Lektion 1 är målet med en linjär regressionsövning att kunna rita en *linje* *som* *passar bäst* för att:

-   **Visa samband mellan variabler**. Visa relationen mellan variabler.

-   **Göra förutsägelser**. Göra exakta förutsägelser om var en ny datapunkt skulle hamna i förhållande till den linjen.

För att rita denna typ av linje använder vi en statistisk teknik som kallas **Minsta kvadratmetoden**. Termen `minsta kvadrat` betyder att alla datapunkter runt regressionslinjen kvadreras och sedan summeras. Idealiskt sett är den slutliga summan så liten som möjligt, eftersom vi vill ha ett lågt antal fel, eller `minsta kvadrat`. Därför är den linje som passar bäst den linje som ger oss det lägsta värdet för summan av de kvadrerade felen - därav namnet *minsta kvadratmetoden*.

Vi gör detta eftersom vi vill modellera en linje som har den minsta kumulativa avvikelsen från alla våra datapunkter. Vi kvadrerar också termerna innan vi summerar dem eftersom vi är intresserade av deras storlek snarare än deras riktning.

> **🧮 Visa mig matematiken**
>
> Denna linje, kallad *linjen som passar bäst*, kan uttryckas med [en ekvation](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` är den '`förklarande variabeln` eller `prediktorn`'. `Y` är den '`beroende variabeln` eller `utfallet`'. Lutningen på linjen är `b` och `a` är skärningspunkten med y-axeln, vilket hänvisar till värdet av `Y` när `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "lutning = $y/x$")
    Infografik av Jen Looper
>
> Först, beräkna lutningen `b`.
>
> Med andra ord, och med hänvisning till vår pumpadata och den ursprungliga frågan: "förutsäg priset på en pumpa per skäppa beroende på månad", skulle `X` hänvisa till priset och `Y` hänvisa till försäljningsmånaden.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Infografik av Jen Looper
> 
> Beräkna värdet av Y. Om du betalar runt \$4, måste det vara april!
>
> Matematiken som beräknar linjen måste visa lutningen på linjen, som också beror på skärningspunkten, eller var `Y` befinner sig när `X = 0`.
>
> Du kan se metoden för att beräkna dessa värden på webbplatsen [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Besök också [denna Minsta kvadrat-kalkylator](https://www.mathsisfun.com/data/least-squares-calculator.html) för att se hur värdena påverkar linjen.

Inte så skrämmande, eller hur? 🤓

#### Korrelation

Ett annat begrepp att förstå är **Korrelationskoefficienten** mellan givna X- och Y-variabler. Med hjälp av ett spridningsdiagram kan du snabbt visualisera denna koefficient. Ett diagram med datapunkter som är snyggt uppradade har hög korrelation, men ett diagram med datapunkter spridda överallt mellan X och Y har låg korrelation.

En bra linjär regressionsmodell är en som har en hög (närmare 1 än 0) korrelationskoefficient med hjälp av Minsta kvadratmetoden och en regressionslinje.


## **2. En dans med data: skapa en data frame som ska användas för modellering**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Konstverk av @allison_horst</figcaption>


<!--![Konstverk av \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Ladda upp nödvändiga bibliotek och dataset. Konvertera data till en data frame som innehåller ett urval av data:

-   Ta endast med pumpor som är prissatta per skäppa

-   Konvertera datumet till en månad

-   Beräkna priset som ett genomsnitt av högsta och lägsta pris

-   Konvertera priset för att återspegla prissättningen per skäppmängd

> Vi gick igenom dessa steg i [föregående lektion](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

I äkta äventyrsanda, låt oss utforska [`janitor-paketet`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) som erbjuder enkla funktioner för att undersöka och rengöra smutsiga data. Till exempel, låt oss titta på kolumnnamnen för våra data:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Vi kan göra bättre. Låt oss göra dessa kolumnnamn `friendR` genom att konvertera dem till [snake_case](https://en.wikipedia.org/wiki/Snake_case)-konventionen med hjälp av `janitor::clean_names`. För att ta reda på mer om denna funktion: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Mycket tidyR 🧹! Nu, en dans med datan med hjälp av `dplyr`, precis som i den förra lektionen! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Bra jobbat! 👌 Du har nu en ren och prydlig datamängd som du kan använda för att bygga din nya regressionsmodell!

Vad sägs om ett spridningsdiagram?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Ett spridningsdiagram påminner oss om att vi bara har månadsdata från augusti till december. Vi behöver troligen mer data för att kunna dra slutsatser på ett linjärt sätt.

Låt oss titta på våra modelleringsdata igen:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Vad händer om vi ville förutsäga `price` för en pumpa baserat på kolumnerna `city` eller `package`, som är av typen text? Eller ännu enklare, hur skulle vi kunna hitta korrelationen (som kräver att båda dess indata är numeriska) mellan till exempel `package` och `price`? 🤷🤷

Maskininlärningsmodeller fungerar bäst med numeriska egenskaper snarare än textvärden, så du behöver vanligtvis konvertera kategoriska egenskaper till numeriska representationer.

Detta innebär att vi måste hitta ett sätt att omforma våra prediktorer för att göra dem enklare för en modell att använda effektivt, en process som kallas `feature engineering`.


## 3. Förbereda data för modellering med recipes 👩‍🍳👨‍🍳

Aktiviteter som omformar prediktorvärden för att göra dem enklare för en modell att använda effektivt kallas `feature engineering`.

Olika modeller har olika krav på förbehandling. Till exempel kräver minsta kvadratmetoden `kodning av kategoriska variabler` såsom månad, sort och stad_namn. Detta innebär helt enkelt att `översätta` en kolumn med `kategoriska värden` till en eller flera `numeriska kolumner` som ersätter den ursprungliga.

Till exempel, anta att din data innehåller följande kategoriska variabel:

|  stad   |
|:-------:|
| Denver  |
| Nairobi |
|  Tokyo  |

Du kan använda *ordinal kodning* för att ersätta varje kategori med ett unikt heltalsvärde, som detta:

| stad |
|:----:|
|  0   |
|  1   |
|  2   |

Och det är precis vad vi ska göra med vår data!

I det här avsnittet ska vi utforska ett annat fantastiskt Tidymodels-paket: [recipes](https://tidymodels.github.io/recipes/) - som är utformat för att hjälpa dig att förbehandla din data **innan** du tränar din modell. Kärnan i en recipe är ett objekt som definierar vilka steg som ska tillämpas på en dataset för att göra den redo för modellering.

Nu ska vi skapa en recipe som förbereder vår data för modellering genom att ersätta varje observation i prediktorkolumnerna med ett unikt heltal:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Fantastiskt! 👏 Vi har precis skapat vårt första recept som specificerar ett utfall (pris) och dess motsvarande prediktorer, och att alla prediktorkolumner ska kodas om till en uppsättning heltal 🙌! Låt oss snabbt bryta ner det:

-   Anropet till `recipe()` med en formel berättar för receptet vilka *roller* variablerna har, med hjälp av `new_pumpkins`-data som referens. Till exempel har kolumnen `price` tilldelats rollen `outcome`, medan resten av kolumnerna har tilldelats rollen `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` specificerar att alla prediktorer ska konverteras till en uppsättning heltal, där numreringen börjar på 0.

Vi är säkra på att du kanske tänker något i stil med: "Det här är så häftigt!! Men vad händer om jag behöver bekräfta att recepten gör exakt det jag förväntar mig? 🤔"

Det är en fantastisk tanke! Du förstår, när ditt recept är definierat kan du uppskatta de parametrar som krävs för att faktiskt förbehandla datan och sedan extrahera den bearbetade datan. Du behöver vanligtvis inte göra detta när du använder Tidymodels (vi kommer att se den normala konventionen om en liten stund -> `workflows`), men det kan vara användbart när du vill göra någon form av rimlighetskontroll för att bekräfta att recepten gör det du förväntar dig.

För detta behöver du två ytterligare verb: `prep()` och `bake()`, och som alltid hjälper våra små R-vänner från [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) dig att förstå detta bättre!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Konstverk av @allison_horst</figcaption>


<!--![Konstverk av \@allison_horst](../../../../../../2-Regression/3-Linear/images/recipes.png){width="550"}-->


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): beräknar de nödvändiga parametrarna från en träningsuppsättning som senare kan tillämpas på andra datamängder. Till exempel, för en given prediktorkolumn, vilken observation kommer att tilldelas heltalet 0, 1, 2 osv.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): tar ett förberett recept och tillämpar operationerna på vilken datamängd som helst.

Med det sagt, låt oss förbereda och tillämpa våra recept för att verkligen bekräfta att bakom kulisserna kommer prediktorkolumnerna först att kodas innan en modell anpassas.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Woo-hoo!🥳 Den bearbetade datan `baked_pumpkins` har alla sina prediktorer kodade, vilket bekräftar att de förbehandlingssteg som definierats som vårt recept fungerar som förväntat. Detta gör det svårare för dig att läsa men mycket mer begripligt för Tidymodels! Ta lite tid att ta reda på vilken observation som har mappats till ett motsvarande heltal.

Det är också värt att nämna att `baked_pumpkins` är en data frame som vi kan utföra beräkningar på.

Till exempel, låt oss försöka hitta en bra korrelation mellan två punkter i din data för att potentiellt bygga en bra prediktiv modell. Vi kommer att använda funktionen `cor()` för detta. Skriv `?cor()` för att ta reda på mer om funktionen.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Som det visar sig finns det bara en svag korrelation mellan Stad och Pris. Däremot finns det en något bättre korrelation mellan Paket och dess Pris. Det är väl logiskt, eller hur? Vanligtvis, ju större lådan med varor, desto högre pris.

När vi ändå håller på, låt oss också försöka visualisera en korrelationsmatris för alla kolumner med hjälp av paketet `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Mycket bättre.

En bra fråga att ställa om dessa data är: '`Vilket pris kan jag förvänta mig för ett visst pumpapaket?`' Låt oss sätta igång!

> Note: När du **`bake()`** det förberedda receptet **`pumpkins_prep`** med **`new_data = NULL`**, extraherar du den bearbetade (dvs. kodade) träningsdatan. Om du hade en annan dataset, till exempel en testuppsättning, och ville se hur ett recept skulle förbehandla den, skulle du helt enkelt baka **`pumpkins_prep`** med **`new_data = test_set`**

## 4. Bygg en linjär regressionsmodell

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografik av Dasani Madipalli</figcaption>


<!--![Infografik av Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Nu när vi har skapat ett recept och faktiskt bekräftat att datan kommer att förbehandlas korrekt, låt oss nu bygga en regressionsmodell för att besvara frågan: `Vilket pris kan jag förvänta mig för ett givet pumpapaket?`

#### Träna en linjär regressionsmodell med träningsuppsättningen

Som du kanske redan har räknat ut är kolumnen *price* den `beroende` variabeln medan kolumnen *package* är den `oberoende` variabeln.

För att göra detta kommer vi först att dela upp datan så att 80% går till träningsuppsättningen och 20% till testuppsättningen, sedan definiera ett recept som kommer att koda den oberoende kolumnen till en uppsättning heltal, och därefter bygga en modellspecifikation. Vi kommer inte att förbereda och baka vårt recept eftersom vi redan vet att det kommer att förbehandla datan som förväntat.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Bra jobbat! Nu när vi har ett recept och en modellspecifikation behöver vi hitta ett sätt att kombinera dem till ett objekt som först förbehandlar data (prep+bake bakom kulisserna), tränar modellen på den förbehandlade datan och även möjliggör eventuella efterbehandlingsaktiviteter. Hur låter det för din sinnesro!🤩

I Tidymodels kallas detta praktiska objekt för en [`workflow`](https://workflows.tidymodels.org/) och innehåller smidigt dina modelleringskomponenter! Det är vad vi skulle kalla *pipelines* i *Python*.

Så låt oss paketera allt i ett workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

Dessutom kan ett arbetsflöde anpassas/tränas på ungefär samma sätt som en modell kan.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Från modellens output kan vi se de koefficienter som lärdes under träningen. De representerar koefficienterna för den bästa linjen som ger oss det lägsta totala felet mellan den faktiska och den förutspådda variabeln.

#### Utvärdera modellens prestanda med hjälp av testuppsättningen

Det är dags att se hur modellen presterade 📏! Hur gör vi detta?

Nu när vi har tränat modellen kan vi använda den för att göra förutsägelser för test_set med `parsnip::predict()`. Därefter kan vi jämföra dessa förutsägelser med de faktiska etikettvärdena för att utvärdera hur bra (eller inte!) modellen fungerar.

Låt oss börja med att göra förutsägelser för testuppsättningen och sedan binda kolumnerna till testuppsättningen.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Ja, du har precis tränat en modell och använt den för att göra förutsägelser! 🔮 Är den bra? Låt oss utvärdera modellens prestanda!

I Tidymodels gör vi detta med `yardstick::metrics()`! För linjär regression fokuserar vi på följande mått:

-   `Root Mean Square Error (RMSE)`: Kvadratroten av [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Detta ger ett absolut mått i samma enhet som etiketten (i detta fall priset på en pumpa). Ju mindre värde, desto bättre modell (förenklat sett representerar det det genomsnittliga priset med vilket förutsägelserna är fel!)

-   `Coefficient of Determination (vanligtvis känt som R-squared eller R2)`: Ett relativt mått där ett högre värde innebär en bättre passform för modellen. I grund och botten representerar detta mått hur mycket av variansen mellan förutsagda och faktiska etikettvärden modellen kan förklara.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Där går modellens prestanda. Låt oss se om vi kan få en bättre indikation genom att visualisera ett spridningsdiagram av paketet och priset, och sedan använda de gjorda förutsägelserna för att lägga till en linje som bäst passar.

Detta innebär att vi måste förbereda och bearbeta testuppsättningen för att koda paketkolumnen och sedan binda detta till de förutsägelser som vår modell har gjort.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Bra! Som du kan se, generaliserar inte linjär regressionsmodellen särskilt väl relationen mellan ett paket och dess motsvarande pris.

🎃 Grattis, du har precis skapat en modell som kan hjälpa till att förutsäga priset på några olika sorters pumpor. Din pumpaplantage inför högtiden kommer att bli fantastisk. Men du kan förmodligen skapa en ännu bättre modell!

## 5. Bygg en polynomregressionsmodell

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografik av Dasani Madipalli</figcaption>


<!--![Infografik av Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Ibland kanske våra data inte har en linjär relation, men vi vill ändå förutsäga ett resultat. Polynomregression kan hjälpa oss att göra förutsägelser för mer komplexa icke-linjära samband.

Ta till exempel sambandet mellan förpackning och pris i vår dataset med pumpor. Även om det ibland finns en linjär relation mellan variabler - ju större pumpan är i volym, desto högre pris - kan dessa samband ibland inte plottas som ett plan eller en rak linje.

> ✅ Här är [några fler exempel](https://online.stat.psu.edu/stat501/lesson/9/9.8) på data som kan använda polynomregression
>
> Titta en gång till på sambandet mellan Sort och Pris i det tidigare diagrammet. Verkar det som att detta spridningsdiagram nödvändigtvis bör analyseras med en rak linje? Kanske inte. I detta fall kan du prova polynomregression.
>
> ✅ Polynom är matematiska uttryck som kan bestå av en eller flera variabler och koefficienter

#### Träna en polynomregressionsmodell med hjälp av träningsdata

Polynomregression skapar en *kurvad linje* för att bättre passa icke-linjär data.

Låt oss se om en polynommodell presterar bättre när det gäller att göra förutsägelser. Vi kommer att följa en liknande procedur som vi gjorde tidigare:

-   Skapa ett recept som specificerar de förbehandlingssteg som ska utföras på våra data för att göra dem redo för modellering, dvs: kodning av prediktorer och beräkning av polynom av grad *n*

-   Bygga en modellspecifikation

-   Paketera receptet och modellspecifikationen i ett arbetsflöde

-   Skapa en modell genom att passa arbetsflödet

-   Utvärdera hur väl modellen presterar på testdata

Nu kör vi igång!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Utvärdera modellens prestanda

👏👏Du har skapat en polynommodell, låt oss göra förutsägelser på testuppsättningen!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo, låt oss utvärdera hur modellen presterade på test_set med hjälp av `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Mycket bättre prestanda.

`rmse` minskade från cirka 7 till cirka 3, vilket indikerar en minskad felmarginal mellan det faktiska priset och det förutspådda priset. Du kan *ungefärligt* tolka detta som att felaktiga förutsägelser i genomsnitt är fel med cirka 3 dollar. `rsq` ökade från cirka 0,4 till 0,8.

Alla dessa mätvärden visar att den polynomiska modellen presterar mycket bättre än den linjära modellen. Bra jobbat!

Låt oss se om vi kan visualisera detta!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Du kan se en kurva som passar dina data bättre! 🤩

Du kan göra detta ännu mjukare genom att skicka en polynomformel till `geom_smooth` så här:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Mycket som en jämn kurva!🤩

Så här gör du en ny förutsägelse:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


Prediktionen med `polynomial model` är rimlig, med tanke på spridningsdiagrammen för `price` och `package`! Och om detta är en bättre modell än den tidigare, baserat på samma data, behöver du planera för dessa dyrare pumpor!

🏆 Bra jobbat! Du skapade två regressionsmodeller under en lektion. I den sista delen om regression kommer du att lära dig om logistisk regression för att bestämma kategorier.

## **🚀Utmaning**

Testa flera olika variabler i denna notebook för att se hur korrelationen påverkar modellens noggrannhet.

## [**Quiz efter föreläsningen**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Granskning & Självstudier**

I denna lektion lärde vi oss om linjär regression. Det finns andra viktiga typer av regression. Läs om Stepwise, Ridge, Lasso och Elasticnet-tekniker. En bra kurs att studera för att lära dig mer är [Stanford Statistical Learning course](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Om du vill lära dig mer om hur du använder det fantastiska Tidymodels-ramverket, kolla in följande resurser:

-   Tidymodels webbplats: [Kom igång med Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn och Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **TACK TILL:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) för att ha skapat de fantastiska illustrationerna som gör R mer välkomnande och engagerande. Hitta fler illustrationer i hennes [galleri](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, vänligen notera att automatiska översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
